In [1]:
%pip install --upgrade datasets transformers sentence-transformers

  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
     ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
      --------------------------------------- 0.1/7.2 MB 1.0 MB/s eta 0:00:07
      --------------------------------------- 0.2/7.2 MB 919.0 kB/s eta 0:00:08
     - -------------------------------------- 0.2/7.2 MB 888.4 kB/s eta 0:00:08
     - -------------------------------------- 0.2/7.2 MB 919.0 kB/s eta 0:00:08
     - -------------------------------------- 0.2/7.2 MB 758.5 kB/s eta 0:00:10
     - -------------------------------------- 0.3/7.2 MB 684.6 kB/s eta 0:00:11
     - -------------------------------------- 0.3/7.2 MB 684.6 kB/s eta 0:00:11
     - -------------------------------------- 0.3/7.2 MB 684.6 kB/s eta 0:00:11
     - -------------------------------------- 0.3/7.2 MB 567.7 kB/s eta 0:00:13
     - -----


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ------- ----------------------------- 34.6/172.3 MB 101.5 kB/s eta 0:22:38
     ------- ----------------------------- 34.6/172.3 MB 101.5 kB/s eta 0:22:38
     ------- ----------------------------- 34.6/172.3 MB 101.5 kB/s eta 0:22:38
     ------- ----------------------------- 34.6/172.3 MB 101.5 kB/s eta 0:22:38
     ------- ----------------------------- 34.6/172.3 MB 101.5 kB/s eta 0:22:38
     ------- ----------------------------- 34.6/172.3 MB 101.5 kB/s eta 0:22:38
     ------- ------------------------------ 34.6/172.3 MB 99.6 kB/s eta 0:23:03
     ------- ------------------------------ 34.6/172.3 MB 99.6 kB/s eta 0:23:03
     ------- ------------------------------ 34.6/172.3 MB 99.6 kB/s eta 0:23:03
     ------- ------------------------------ 34.6/172.3 MB 99.6 kB/s eta 0:23:03
     ------- ----------------------------- 34.6/172.3 MB 101.0 kB/s eta 0:22:44
     ------- ----------------------------- 34.6/172.3 MB 101.0 kB/s eta 0:22:44
     ------- --------------------------

In [2]:
%pip install -qU git+https://github.com/naver/splade.git



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install -qU pinecone-client
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/635.3 kB ? eta -:--:--
     ----------- -------------------------- 194.6/635.3 kB 4.0 MB/s eta 0:00:01
     ------------------------ ------------- 409.6/635.3 kB 4.3 MB/s eta 0:00:01
     -------------------------------------- 635.3/635.3 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# !pip install -qU tiktoken

## Dataset Preparation


In [5]:
import os
import json

def crawl_directory(folder_path):
    res = []
    for r, d, f in os.walk(folder_path):
        for directory in d:
                res.append(os.path.join(r, directory))
    return res

path_ls = crawl_directory("../../raw_domo_kb/article/")
path_ls[0:10]

['../../raw_domo_kb/article/000005048',
 '../../raw_domo_kb/article/000005059',
 '../../raw_domo_kb/article/000005073',
 '../../raw_domo_kb/article/000005080',
 '../../raw_domo_kb/article/000005090',
 '../../raw_domo_kb/article/000005091',
 '../../raw_domo_kb/article/000005095',
 '../../raw_domo_kb/article/000005099',
 '../../raw_domo_kb/article/000005100',
 '../../raw_domo_kb/article/000005103']

In [6]:
import os
import json

def trim_newline(string):
    return string.replace("\n\n\n", '')

def load_dataset(path_ls):
    dataset = []
    for file_path in path_ls:
        process_path =os.path.join(file_path, 'process.json') 
        if not os.path.exists(process_path):
            continue
        with open(process_path, encoding="utf-8") as f:
            data = json.loads(f.read())
            article = {'title': trim_newline(data['content']['title']),
                            'id': trim_newline(data['content']['article_number']),
                            }

        md_path = os.path.join(file_path, 'index.md')        
        with open(md_path, encoding = "utf-8") as f:
            article_md = f.read()
            article.update({'article': article_md.replace('\n\n', '\n')})
         

        dataset.append(article)
    return dataset

kb_ds = load_dataset(path_ls)


In [7]:
import pandas as pd
print(f"num_rows: {len(kb_ds)}")
pd.DataFrame(kb_ds[0:5])


num_rows: 1773


,title,id,article
0,Flex Map v2 User Guide,000005048,\nIntro\n-------\nFlex Map v2 is a premium app...
1,Domo Sandbox: Linked Repositories,000005059,"\nIntro\n-------\nIn Domo Sandbox, linked repo..."
2,Flex Map v2 Implementation Guide,000005073,\nIntro\n-------\n\n**Important:**\n This guid...
3,Update and Switch Card Assets,000005080,\nIntro\n-------\nThe\n **Asset Library**\n st...
4,e-Stat Third Party Connector,000005090,\nIntro\n-------\ne-Stat is the one-stop servi...


We need to cut our contexts into digestable chunks for our models. We'll be using BERT which has a max sequence length of `512` tokens, _but_ typical sentence transformers limit this to `128`.

To be safe and ensure we're not over the smaller `128` token limit we will assume an average token length of `3` characters (in reality it is more like _3-5_) and therefore our required length will be `128*3 == 384` characters.

To build passages of this length we will define a processing function called `article_chunker`.


In [8]:
import tiktoken
from langchain.text_splitter import MarkdownTextSplitter

TOKEN_CHUNK_SIZE = 384

article_text_splitter = MarkdownTextSplitter(
    chunk_size=TOKEN_CHUNK_SIZE,
    chunk_overlap=20,  # number of tokens overlap between chunks
)

def article_chunker(article, article_text_splitter): 
    doc_chunk = article_text_splitter.create_documents([article['article']], metadatas=[{
                                        'article_id': article['id'], 'title': article['title']}])

    for index,  chunk in enumerate(doc_chunk):
        chunk.metadata.update({'id' :  f"{chunk.metadata['article_id']}-{index}"})
    
    return doc_chunk

data = article_chunker(kb_ds[0], article_text_splitter)
data[0:10]

ModuleNotFoundError: No module named 'tiktoken'

## Model Initialization and Vectors

With our dataset prepared we can move on to initializing the required models and setting up some helper functions to make sparse and dense vector building easy.

### Dense Vectors

Starting with the dense vectors, we will use an off-the-shelf model from the `sentence-transformers` library.


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# check device being run on
if device != 'cuda':
    print("==========\n" +
          "WARNING: You are not running on GPU so this may be slow.\n" +
          "If on Google Colab, go to top menu > Runtime > Change " +
          "runtime type > Hardware accelerator > 'GPU' and rerun " +
          "the notebook.\n==========")

dense_model = SentenceTransformer(
    'msmarco-bert-base-dot-v5',
    device=device
)
dense_model


We then create an embedding very easily like so:


In [ ]:
emb = dense_model.encode(data[2].page_content)
emb.shape


The model returns `768` dimensional dense vectors, this is also reflected in the model attributes.


In [ ]:
dim = dense_model.get_sentence_embedding_dimension()
dim


### Sparse Vectors

We will also need to create sparse vectors. For that we will be using a learned sparse embedding model called SPLADE. SPLADE actually consists of many models that use similar embedding methods, we will be using the `naver/splade-cocondenser-ensembledistil` model.

We initialize the model like so:


In [ ]:
from splade.models.transformer_rep import Splade

sparse_model_id = 'naver/splade-cocondenser-ensembledistil'

sparse_model = Splade(sparse_model_id, agg='max')
sparse_model.to(device)  # move to GPU if possible
sparse_model.eval()


The model takes tokenized inputs that are built using a tokenizer initialized with the same model ID.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(sparse_model_id)

tokens = tokenizer(data[0]['context'], return_tensors='pt')


To create sparse vectors we do:


In [ ]:
with torch.no_grad():
    sparse_emb = sparse_model(
        d_kwargs=tokens.to(device)
    )['d_rep'].squeeze()
sparse_emb.shape


In [ ]:
sparse_emb


Leaving us with a `30522` dimensional sparse vector embedding. Pinecone will expect a dictionary style format of the sparse vector. To build it we take a couple more steps.

First we get a list of non-zero positions in the vector.


In [ ]:
indices = sparse_emb.nonzero().squeeze().cpu().tolist()
print(len(indices))


We have `174` non-zero values, we use them to create a dictionary of index positions to scores like so:


In [ ]:
values = sparse_emb[indices].cpu().tolist()
sparse = {'indices': indices, 'values': values}
sparse


This is the format that Pinecone requires, from here we can move on to indexing. We'll take a brief detour to explore the sparse vectors, but if you'd rather jump straight to indexing, skip the next section.

---

#### Reading Sparse Embedding

But moving onto indexing everything in Pinecone, let's take a moment to understand what our sparse vector means. We can translate these into a human readable format so we can see what this sparse vector is actually representing.

We create a way of mapping from index positions to actual BERT tokenizer tokens.


In [ ]:
idx2token = {idx: token for token, idx in tokenizer.get_vocab().items()}


Then create the mappings like we did with the Pinecone-friendly sparse format above.


In [ ]:
sparse_dict_tokens = {
    idx2token[idx]: round(weight, 2) for idx, weight in zip(indices, values)
}
# sort so we can see most relevant tokens first
sparse_dict_tokens = {
    k: v for k, v in sorted(
        sparse_dict_tokens.items(),
        key=lambda item: item[1],
        reverse=True
    )
}
sparse_dict_tokens


---

## Indexing Everything

To build the vector DB we will need to index everything, for this we will need to initialize our connection to Pinecone, create an index, and insert everything in the format:

```python
{
    'id': 'id-123',
    'values': [0.1, 0.2, ...],  # dense vec
    'sparse_values': {
        'indices': [23, 718],
        'values': [0.25, 0.77]
    },  # sparse vec
    'metadata': {"context": "some text here"}  # metadata dict
}
```

To make things easier we can create a helper function to transform a list of records from `data` into this format, we'll call it `builder`:


In [ ]:
import pinecone


def builder(records: list):
    ids = [x['id'] for x in records]
    contexts = [x['context'] for x in records]
    # create dense vecs
    dense_vecs = dense_model.encode(contexts).tolist()
    # create sparse vecs
    input_ids = tokenizer(
        contexts, return_tensors='pt',
        padding=True, truncation=True
    )
    with torch.no_grad():
        sparse_vecs = sparse_model(
            d_kwargs=input_ids.to(device)
        )['d_rep'].squeeze()
    # convert to upsert format
    upserts = []
    for _id, dense_vec, sparse_vec, context in zip(ids, dense_vecs, sparse_vecs, contexts):
        # extract columns where there are non-zero weights
        indices = sparse_vec.nonzero().squeeze().cpu().tolist()  # positions
        values = sparse_vec[indices].cpu().tolist()  # weights/scores
        # build sparse values dictionary
        sparse_values = {
            "indices": indices,
            "values": values
        }
        # build metadata struct
        metadata = {'context': context}
        # append all to upserts list as pinecone.Vector (or GRPCVector)
        upserts.append({
            'id': _id,
            'values': dense_vec,
            'sparse_values': sparse_values,
            'metadata': metadata
        })
    return upserts


In [ ]:
builder(data[:3])


Now we initialize our connection to Pinecone using a [free API key](https://app.pinecone.io/).


In [ ]:
import pinecone

pinecone.init(
    api_key="YOUR_API_KEY",  # app.pinecone.io
    environment="YOUR_ENV"  # next to api key in console
)


Then create a new sparse-dense enabled index (this requires `metric="dotproduct"` and `pod_type` to be `p1` or `s1`.


In [ ]:
index_name = 'pubmed-splade'

pinecone.create_index(
    index_name,
    dimension=dim,
    metric="dotproduct",
    pod_type="s1"
)


Initialize with `GRPCIndex` or `Index`:


In [ ]:
index = pinecone.GRPCIndex(index_name)
index.describe_index_stats()


Upsert to sparse-dense is simple:


In [ ]:
index.upsert(builder(data[:3]))


We can repeat this and iterate through (and index) the full dataset:


In [ ]:
from tqdm.auto import tqdm

batch_size = 64

for i in tqdm(range(0, len(data), batch_size)):
    # extract batch of data
    i_end = min(i+batch_size, len(data))
    batch = data[i:i_end]
    # pass data to builder and upsert
    index.upsert(builder(data[i:i+batch_size]))


We can check the number of upserted records aligns with the length of `data`.


In [ ]:
len(data), index.describe_index_stats()


And now we can move on to querying...

## Queries

Our queries need to contain both dense and sparse vectors, we will define a function `encode` to handle the construction of vectors from text.


In [ ]:
def encode(text: str):
    # create dense vec
    dense_vec = dense_model.encode(text).tolist()
    # create sparse vec
    input_ids = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        sparse_vec = sparse_model(
            d_kwargs=input_ids.to(device)
        )['d_rep'].squeeze()
    # convert to dictionary format
    indices = sparse_vec.nonzero().squeeze().cpu().tolist()
    values = sparse_vec[indices].cpu().tolist()
    sparse_dict = {"indices": indices, "values": values}
    # return vecs
    return dense_vec, sparse_dict


In [ ]:
query = "Can clinicians use the PHQ-9 to assess depression in people with vision loss?"
dense, sparse = encode(query)
# query
xc = index.query(
    vector=dense,
    sparse_vector=sparse,
    top_k=2,  # how many results to return
    include_metadata=True
)
xc


We get three answers that talk about **PHQ-9** and **depression**, but only the first result is relevant to our specific scenario of these items for people with **vision loss**. This is a strong result. However, how much of this is down to the sparse vs dense components?

We can actually modify the dense and sparse vectors being used to query, so that we _"weight"_ one or the other to be more/less important.

We set a scaling value `alpha` and implement it so that when `alpha == 0` we are doing a pure **sparse** search, and when `alpha == 1` we are doing a pure **dense** search.


In [ ]:
def hybrid_scale(dense, sparse, alpha: float):
    # check alpha value is in range
    if alpha < 0 or alpha > 1:
        raise ValueError("Alpha must be between 0 and 1")
    # scale sparse and dense vectors to create hybrid search vecs
    hsparse = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    hdense = [v * alpha for v in dense]
    return hdense, hsparse


Let's try a pure dense search:


In [ ]:
hdense, hsparse = hybrid_scale(dense, sparse, alpha=1.0)
# query
xc = index.query(
    vector=hdense,
    sparse_vector=hsparse,
    top_k=2,  # how many results to return
    include_metadata=True
)
xc


Dense is actually performing very well here, let's try a full sparse search:


In [ ]:
hdense, hsparse = hybrid_scale(dense, sparse, alpha=0.0)
# query
xc = index.query(
    vector=hdense,
    sparse_vector=hsparse,
    top_k=2,  # how many results to return
    include_metadata=True
)
xc


In this scenario, both models are returning good results. But we can see variation in the results and their respective scores when switching between more or less sparse-vs-dense.


In [ ]:
query = "Does ibuprofen increase perioperative blood loss during hip arthroplasty?"
dense, sparse = encode(query)
hdense, hsparse = hybrid_scale(dense, sparse, alpha=0.0)  # pure SPARSE
# query
xc = index.query(
    vector=hdense,
    sparse_vector=hsparse,
    top_k=2,  # how many results to return
    include_metadata=True
)
xc


Here the term `hip arthroplasty` refers to a hip replacement. Using Splade we get the answer ranked at #1. Let's try with dense:


In [ ]:
query = "Does ibuprofen increase perioperative blood loss during hip arthroplasty?"
dense, sparse = encode(query)
hdense, hsparse = hybrid_scale(dense, sparse, alpha=1.0)  # pure DENSE
# query
xc = index.query(
    vector=hdense,
    sparse_vector=hsparse,
    top_k=2,  # how many results to return
    include_metadata=True
)
xc


Using dense only we still get the "best" answer at #2, but we return a less relevant answer first. For this question and others, SLADE can outperform dense models, particularly those that haven't been fine-tuned on the data source.

Naturally, searching with both sparse and dense can give us the best of both worlds. We give important to keyword matches, without losing the semantic meaning component.


Try some more questions!

Here are a few ideas [sourced from the PubMedQA paper](https://aclanthology.org/D19-1259/):

```
Do preoperative statins reduce atrial fibrillation after coronary artery bypass grafting?

Spontaneous electrocardiogram alterations predict ventricular fibrillation in Brugada syndrome

Do liver grafts from selected older donors have significantly more ischaemia reperfusion injury?

Does reducing spasticity translate into functional benefit?

Does ibuprofen increase perioperative blood loss during hip arthroplasty?

Should circumcision be performed in childhood?

Is external palliative radiotherapy for gallbladder carcinoma effective?

Sternal fracture in growing children: A rare and often overlooked fracture?

Xanthogranulomatous cholecystitis: a premalignant condition?

Can PRISM predict length of PICU stay?

Is trabecular bone related to primary stability of miniscrews?
```

Once you're done, delete the index to save resources.


In [ ]:
pinecone.delete_index(index_name)


---
